<a href="https://colab.research.google.com/github/jaggukaka/TTS/blob/main/experiments/yourtts_telugu_openslr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.chdir('/content')
!rm -rf TTS
!git clone https://github.com/jaggukaka/TTS

#!git clone https://github.com/Edresson/Coqui-TTS -b multilingual-torchaudio-SE TTS
!pip install -q -e TTS/

!pip install -q torchaudio==0.9.0

Cloning into 'TTS'...
remote: Enumerating objects: 23746, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 23746 (delta 12), reused 7 (delta 3), pack-reused 23719
Receiving objects: 100% (23746/23746), 218.20 MiB | 274.00 KiB/s, done.
Resolving deltas: 100% (17256/17256), done.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 66 kB 3.9 MB/s 
     |████████████████████████████████| 284 kB 15.7 MB/s 
     |████████████████████████████████| 3.4 MB 78.5 MB/s 
     |████████████████████████████████| 133 kB 77.2 MB/s 
     |████████████████████████████████| 80 kB 12.4 MB/s 
     |████████████████████████████████| 487 kB 70.1 MB/s 
     |████████████████████████████████| 212 kB 79.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


     |████████████████████████████████| 1.9 MB 5.3 MB/s 
     |████████████████████████████████| 831.4 MB 2.5 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.


In [ ]:
import torch
torch.cuda.current_device()

0

In [ ]:
print (torch.cuda.device_count(), torch.cuda.get_device_name(0), torch.cuda.is_available())
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

1 Tesla P100-PCIE-16GB True
Tue Feb 15 08:18:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                            

In [ ]:
! pwd

/content


In [ ]:
import os
os.chdir('TTS')
!pip install -r requirements.txt
!python setup.py develop


/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'build-lib' will not be supported in future versions. Please use the underscore name 'build_lib' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'build-dir' will not be supported in future versions. Please use the underscore name 'build_dir' instead
  % (opt, underscore_opt))
running develop
running egg_info
writing TTS.egg-info/PKG-INFO
writing dependency_links to TTS.egg-info/dependency_links.txt
writing entry points to TTS.egg-info/entry_points.txt
writing requirements to TTS.egg-info/requires.txt
writing top-level names to TTS.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE.txt'
writing manifest file 'TTS.egg-info/SOURCES.txt'
running build_ext
copying build/lib.linux-x86_64-3.7/TTS/tts/utils/monotonic_align/core.cpython-37m-x86_64-linux-gnu.so -> TTS/tts

In [ ]:
os.chdir('/content/TTS')
! git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/jaggukaka/TTS
   b73162e3..9f587b82  main       -> origin/main
Updating b73162e3..9f587b82
Fast-forward
 TTS/trainer.py | 14 +++++++-------
 1 file changed, 7 insertions(+), 7 deletions(-)


In [ ]:
# training code
!pip install --upgrade numpy

     |████████████████████████████████| 15.7 MB 8.7 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
tts 0.5.0 requires numpy==1.19.5, but you have numpy 1.21.5 which is incompatible.


In [ ]:
import os
os.chdir('/content')
import sys
TTS_PATH = "TTS/"

# add libraries into environment
sys.path.append(TTS_PATH) # set this if TTS is not installed globally
! CUDA_VISIBLE_DEVICES="0" python TTS/TTS/bin/train_tts.py --continue_path /content/drive/MyDrive/tts_dubbing_coqui/checkpoints/VITS-multilingual/tts-telugu/openslr/run_vcdemo/vits_tts-telugu-February-15-2022_08+19AM-0000000

Printing the config data  before all processing block2 VitsConfig(model='vits', run_name='vits_tts-telugu', run_description='', epochs=1000, batch_size=15, eval_batch_size=15, mixed_precision=False, scheduler_after_epoch=True, run_eval=True, test_delay_epochs=-1, print_eval=True, dashboard_logger='tensorboard', print_step=15, plot_step=200, model_param_stats=False, project_name=None, log_model_step=5000, wandb_entity=None, save_step=5000, checkpoint=True, keep_all_best=False, keep_after=5000, num_loader_workers=4, num_eval_loader_workers=4, use_noise_augment=False, use_language_weighted_sampler=True, output_path='/content/drive/MyDrive/tts_dubbing_coqui/checkpoints/VITS-multilingual/tts-telugu/openslr/run_vcdemo/', distributed_backend='nccl', distributed_url='tcp://localhost:54321', audio=BaseAudioConfig(fft_size=1024, win_length=1024, hop_length=256, frame_shift_ms=None, frame_length_ms=None, stft_pad_mode='reflect', sample_rate=16000, resample=True, preemphasis=0.0, ref_level_db=20, 